# Advanced Video Colorization with Sequence-Based Model

This notebook demonstrates video colorization using an advanced sequence-based model, conceptually similar to architectures like BiSTNet. Such models consider multiple frames at once to improve temporal consistency and reduce flickering compared to single-frame colorization methods.

## Setup

Import necessary modules and set up the environment.

In [ ]:
#NOTE:  This must be the first DeOldify cell to run! Select GPU environment in Colab for best performance.
from deoldify import device
from deoldify.device_id import DeviceId

#Choices: CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)

In [ ]:
import warnings
from deoldify.visualize import get_advanced_video_colorizer, show_video_in_notebook
import matplotlib.pyplot as plt

# Matplotlib style for dark background, optional
plt.style.use('dark_background')
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

## Initialize the Advanced Video Colorizer

This step loads the sequence-based model. The `n_frames_input` parameter (defaulting to 5 in the model setup) determines how many frames the model considers simultaneously. A higher number can lead to better temporal consistency but may be more computationally intensive.

In [ ]:
# Parameters like weights_name, render_factor, n_frames_input can be adjusted here if needed,
# but we'll use defaults or values passed during get_advanced_video_colorizer setup.
colorizer = get_advanced_video_colorizer() 
# To use specific weights or model parameters (if future models allow):
# colorizer = get_advanced_video_colorizer(weights_name='MyAdvancedModel.pth', n_frames_input=7)

## Instructions

### `source_url`
Link to a video from YouTube, Imgur, Twitter, Reddit, Vimeo, etc. GIFs also work. Full list: [youtube-dl supported sites](https://ytdl-org.github.io/youtube-dl/supportedsites.html).
To use your own video, set `source_url = None` and upload your file to the `video/source/` directory in your Jupyter environment. Ensure `file_name` matches your uploaded file.

### `file_name` (Base name for the video output, without extension)
A sensible base name for your video. If using `source_url`, this will be the prefix for the downloaded and output files. If `source_url` is `None`, this should match the stem of your uploaded file in `video/source/`.

### `render_factor`
Determines the resolution at which the colorization model processes the video. Lower values render faster, and colors might appear more vibrant. Higher values can be better for high-quality input but may wash out colors slightly. The default is 21. Max is ~44 on 11GB GPUs.

### `result_path`
This will be automatically determined. The final colorized video will be in the `video/result/` directory.

## Colorize Video!

In [ ]:
# Default render_factor is 21. Adjust if needed.
render_factor = 21 

# Example using a URL:
source_url = 'https://www.youtube.com/watch?v=your_video_id_here' # Replace with a real video URL
# Base name for the output video file (without extension)
file_name_stem = 'my_advanced_colorized_video'

# To use a local file instead:
# source_url = None 
# file_name_stem = 'my_local_video' # Assuming 'my_local_video.mp4' or similar is in video/source/

if source_url is not None:
    # For URL processing, colorize_from_url now expects a base_file_name (stem)
    result_path = colorizer.colorize_from_url(source_url, base_file_name=file_name_stem, render_factor=render_factor)
else:
    # For local file processing, colorize_from_file_name expects the full name with extension
    # User needs to ensure the file_name_with_ext exists in video/source/
    file_name_with_ext = file_name_stem + '.mp4' # User should adjust extension if not mp4
    print(f"Attempting to colorize local file: video/source/{file_name_with_ext}")
    result_path = colorizer.colorize_from_file_name(file_name_with_ext, render_factor=render_factor)

print(f"Colorized video saved to: {result_path}")
show_video_in_notebook(result_path)

## Discussion & Comparison

The sequence-based model used here is designed to produce more temporally stable colorization with reduced flickering compared to models that process each frame independently. 

Consider colorizing the same video using the original `VideoColorizer.ipynb` (which uses a single-frame model) and compare the results. Pay attention to:
- Flickering in areas that should have consistent color.
- Color consistency of moving objects.

Note: The actual visual improvement depends heavily on the underlying model architecture and training. This notebook provides the *framework* for using such advanced models.

## Frame Preview (Optional)

The `plot_transformed_image` function visualizes how a single frame is colorized. When used with the `VideoColorizerFilter`, this shows the result of its single-image processing mode (which duplicates the frame to form a sequence). This preview might not fully represent the temporal benefits seen in the full video but can be useful for checking general color quality for a given `render_factor`.

In [ ]:
# This uses the .P() method of the filter, which for VideoColorizerFilter duplicates the frame.
# Ensure you have processed a video first so that 'bwframes' are available, or provide a direct image path.

# Example: if you ran the cell above with file_name_stem = 'my_advanced_colorized_video'
# and it downloaded and extracted frames, you might find a frame like:
# 'video/bwframes/my_advanced_colorized_video_downloaded_from_url_ HASH /00001.jpg'
# The exact path will depend on how VideoProcessor names the frame extraction folder.
# Let's assume a known local image for a stable example:
# colorizer.vis.plot_transformed_image('path_to_your_test_image.jpg', render_factor=21, display_render_factor=True, figsize=(8,8))

print("Frame preview for sequence models shows single-frame duplication behavior.")
print("To test frame previews, ensure 'video/bwframes/<file_name_stem>/00001.jpg' exists from a previous run, or point to a valid image.")
# Example, if 'my_advanced_colorized_video' was processed:
# frame_preview_path = 'video/bwframes/' + file_name_stem + '/00001.jpg' 
# if os.path.exists(frame_preview_path):
#   colorizer.vis.plot_transformed_image(frame_preview_path, render_factor=21, display_render_factor=True, figsize=(8,8))
# else:
#   print(f"Preview frame {frame_preview_path} not found. Run video colorization first or provide a direct image path.")